In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [4]:
rng = pd.date_range('1/1/2017', '9/1/2017', freq='WOM-3FRI') # 매주 셋째 주 금요일
list(rng)

[Timestamp('2017-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-03-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-04-21 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-05-19 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-06-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-07-21 00:00:00', freq='WOM-3FRI'),
 Timestamp('2017-08-18 00:00:00', freq='WOM-3FRI')]

* 오프셋만큼 날짜 시프트

In [5]:
from pandas.tseries.offsets import Day, MonthEnd

In [6]:
now = datetime(2017, 9, 1)
now + 3 * Day()

Timestamp('2017-09-04 00:00:00')

In [8]:
now + MonthEnd()

Timestamp('2017-09-30 00:00:00')

In [10]:
now + MonthEnd(2)

Timestamp('2017-10-31 00:00:00')

* rollforward, rollback

In [11]:
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2017-09-30 00:00:00')

In [12]:
offset.rollback(now)

Timestamp('2017-08-31 00:00:00')

In [14]:
ts = pd.Series(np.random.randn(20), index=pd.date_range('1/1/2017', periods=20, freq='4d'))
ts.groupby(offset.rollforward).mean()

2017-01-31    0.026317
2017-02-28   -0.133188
2017-03-31   -0.363750
dtype: float64

In [16]:
ts['2017/01'].mean()

0.026316779216114774

In [18]:
ts.resample('M').mean()

2017-01-31    0.026317
2017-02-28   -0.133188
2017-03-31   -0.363750
Freq: M, dtype: float64

### 시간대 다루기

* 파이썬에서 시간대 정보는 전 세계의 시간대정보를 모아둔 올슨 데이터베이스를 담고 있는 서드파티 라이브러리, pytz에서 얻어옴.

In [19]:
import pytz

In [20]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [22]:
tz = pytz.timezone('US/Eastern')
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

### 지역화와 변환

In [26]:
rng = pd.date_range('3/9/2017 9:30', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2017-03-09 09:30:00   -0.848313
2017-03-10 09:30:00   -1.168855
2017-03-11 09:30:00   -0.673409
2017-03-12 09:30:00    0.202332
2017-03-13 09:30:00   -0.021606
2017-03-14 09:30:00    0.339656
Freq: D, dtype: float64

In [27]:
print(ts.index.tz) # default : None

None


* 시간대를 지정해서 날짜 범위를 생성할 수 있다.

In [28]:
pd.date_range('3/9/2017 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2017-03-09 09:30:00+00:00', '2017-03-10 09:30:00+00:00',
               '2017-03-11 09:30:00+00:00', '2017-03-12 09:30:00+00:00',
               '2017-03-13 09:30:00+00:00', '2017-03-14 09:30:00+00:00',
               '2017-03-15 09:30:00+00:00', '2017-03-16 09:30:00+00:00',
               '2017-03-17 09:30:00+00:00', '2017-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

* 지역화 시간으로의 변환은 tz_localize 메서드를 사용해서 처리할 수 있음

In [29]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2017-03-09 09:30:00+00:00   -0.848313
2017-03-10 09:30:00+00:00   -1.168855
2017-03-11 09:30:00+00:00   -0.673409
2017-03-12 09:30:00+00:00    0.202332
2017-03-13 09:30:00+00:00   -0.021606
2017-03-14 09:30:00+00:00    0.339656
Freq: D, dtype: float64

In [30]:
ts_utc.index

DatetimeIndex(['2017-03-09 09:30:00+00:00', '2017-03-10 09:30:00+00:00',
               '2017-03-11 09:30:00+00:00', '2017-03-12 09:30:00+00:00',
               '2017-03-13 09:30:00+00:00', '2017-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

* 시계열이 특정 시간대로 지역화가 되고 나면 tz_convert를 이용하여 다른 시간대로 변환할 수 있음

In [31]:
ts_utc.tz_convert('US/Eastern')

2017-03-09 04:30:00-05:00   -0.848313
2017-03-10 04:30:00-05:00   -1.168855
2017-03-11 04:30:00-05:00   -0.673409
2017-03-12 05:30:00-04:00    0.202332
2017-03-13 05:30:00-04:00   -0.021606
2017-03-14 05:30:00-04:00    0.339656
Freq: D, dtype: float64

In [32]:
ts_eastern = ts.tz_localize('US/Eastern')
ts_eastern.tz_convert('UTC')

2017-03-09 14:30:00+00:00   -0.848313
2017-03-10 14:30:00+00:00   -1.168855
2017-03-11 14:30:00+00:00   -0.673409
2017-03-12 13:30:00+00:00    0.202332
2017-03-13 13:30:00+00:00   -0.021606
2017-03-14 13:30:00+00:00    0.339656
Freq: D, dtype: float64

In [33]:
ts_eastern.tz_convert('Europe/Berlin')

2017-03-09 15:30:00+01:00   -0.848313
2017-03-10 15:30:00+01:00   -1.168855
2017-03-11 15:30:00+01:00   -0.673409
2017-03-12 14:30:00+01:00    0.202332
2017-03-13 14:30:00+01:00   -0.021606
2017-03-14 14:30:00+01:00    0.339656
Freq: D, dtype: float64

In [34]:
ts_eastern.tz_convert('Asia/Shanghai')

2017-03-09 22:30:00+08:00   -0.848313
2017-03-10 22:30:00+08:00   -1.168855
2017-03-11 22:30:00+08:00   -0.673409
2017-03-12 21:30:00+08:00    0.202332
2017-03-13 21:30:00+08:00   -0.021606
2017-03-14 21:30:00+08:00    0.339656
Freq: D, dtype: float64